In [8]:
from google.colab import files
uploaded = files.upload()

Saving gender_submission.csv to gender_submission.csv


In [9]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.ensemble import RandomForestClassifier as RandomForest
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
GS = pd.read_csv("gender_submission.csv")
all = pd.concat([train, test], axis =0).reset_index(drop=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if sys.path[0] == '':


In [10]:
def kesson_table(df):
    null_val = df.isnull().sum()
    percent = 100 * df.isnull().sum()/len(df)
    kesson_table = pd.concat([null_val, percent], axis=1)
    kesson_table_ren_columns = kesson_table.rename(
        columns = {0 : '欠損数', 1 : '%'})
    return kesson_table_ren_columns

kesson_table(all)

,欠損数,%
Age,263,20.091673
Cabin,1014,77.463713
Embarked,2,0.152788
Fare,1,0.076394
Name,0,0.000000
Parch,0,0.000000
PassengerId,0,0.000000
Pclass,0,0.000000
Sex,0,0.000000
SibSp,0,0.000000


In [0]:
all["surname"] = all["Name"].apply(lambda x:x.split(",")[0])

In [12]:
all['Age'].fillna(all['Age'].mode()[0], inplace=True)
all["Embarked"] = all["Embarked"].fillna("S")
all.Fare[1043] = all.Fare.mean()

kesson_table(all)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,欠損数,%
Age,0,0.000000
Cabin,1014,77.463713
Embarked,0,0.000000
Fare,0,0.000000
Name,0,0.000000
Parch,0,0.000000
PassengerId,0,0.000000
Pclass,0,0.000000
Sex,0,0.000000
SibSp,0,0.000000


In [0]:
cols = ["surname", "Sex","Embarked"]
all_to_cat = pd.get_dummies(all, columns=cols)

In [14]:
data_counts = all['surname'].value_counts()
surname_to_drop = data_counts[data_counts < 3].index.tolist()

to_drop_list = []
target_col = "surname"

for each_data in surname_to_drop:
    col_name = "{}_{}".format(target_col, each_data)
    to_drop_list.append(col_name)
    
all_to_cat.drop(columns=to_drop_list, inplace=True)

to_drop = ["PassengerId", "Name", "Ticket", "Cabin"]

all_to_cat.drop(columns=to_drop, inplace=True)
print(all_to_cat.shape)

(1309, 116)


In [0]:
train_to_cat = all_to_cat.iloc[0:891, :]
train_train = train_to_cat.drop(["Survived"], axis=1)

In [0]:
target = train_to_cat["Survived"].values
features = train_train.values

In [0]:
x_train, x_test, t_train, t_test = train_test_split(features, target, test_size =0.3, random_state = 0)

In [18]:
model = RandomForest(n_estimators=100).fit(x_train, t_train)
model.score(x_test, t_test)

0.835820895522388

In [19]:
header = np.array(train.drop(['Survived', 'PassengerId'], axis=1).columns)
print(header)
print(model.feature_importances_)

['Pclass' 'Name' 'Sex' 'Age' 'SibSp' 'Parch' 'Ticket' 'Fare' 'Cabin'
 'Embarked']
[2.09527976e-01 1.99197761e-01 2.65745758e-02 7.46943036e-02
 4.51628004e-02 1.81913235e-04 5.38911097e-03 7.58205332e-03
 3.13740667e-03 2.09005381e-03 2.49759677e-03 2.62595588e-04
 4.20771804e-03 1.63834538e-03 2.18063676e-03 1.41976126e-03
 6.10016882e-04 4.64242770e-03 4.48839334e-04 5.79284327e-04
 1.88663227e-04 0.00000000e+00 1.86095773e-03 6.67079282e-04
 6.10825915e-03 1.15263606e-03 1.22383112e-03 6.38780098e-05
 1.22892555e-03 1.22608715e-03 1.26747878e-04 8.25234648e-04
 2.20846614e-03 3.39144367e-03 1.50543636e-03 2.27383751e-03
 2.14450563e-04 3.28351823e-03 1.47538495e-03 8.25204709e-04
 4.21850714e-04 1.82655281e-04 1.81202376e-03 1.37231903e-04
 3.98394603e-04 0.00000000e+00 4.28557402e-04 6.24551563e-04
 5.06050657e-04 0.00000000e+00 2.31438229e-03 2.65381947e-04
 2.37198763e-04 1.12699857e-03 1.57018415e-03 4.40459025e-03
 9.40964309e-05 5.47250876e-04 7.20612762e-04 1.14357764e-03
 0.

In [0]:
test_to_cat = all_to_cat.iloc[891:, :]
test_test = test_to_cat.drop(["Survived"], axis=1)

In [0]:
test_features = test_test.values

In [22]:
preds = model.predict(test_features)
preds

array([0., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0.,
       1., 0., 0., 1., 0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 1., 0.,
       1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 0., 1., 1., 0.,
       0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 1., 1., 0.,
       0., 1., 1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0.,
       0., 1., 0., 1., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0.,
       1., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0.,
       1., 0., 0., 1., 0., 1., 1., 1., 1., 1., 0., 0., 1., 0., 0., 1., 1.,
       0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 1., 1., 1.,
       0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0., 1.,
       0., 1., 0., 0., 1.

In [0]:
PassengerId = np.array(test["PassengerId"]).astype(int)
                            
my_solution = pd.DataFrame(preds, PassengerId, columns = ["Survived"])
                            
my_solution.to_csv("GRY2.csv", index_label = ["PassengerId"])

In [0]:
from google.colab import files
files.download("GRY2.csv")

73.2%に落ちた。どうやら苗字は微妙。